## gexf graphs

In [1]:
# for python 2.7
import joblib
import os
import networkx as nx
import time
import ast
from random import shuffle
import pandas as pd

In [ ]:
def count_non_unique_names(dic, non_uniques={}):
    for key in dic:
        if key in non_uniques:
                non_uniques[key] += 1
        else:
            non_uniques[key] = 0
        if type(dic[key]) is dict:
            count_non_unique_names(dic[key], non_uniques)
    res = {}
    for key in non_uniques:
        if non_uniques[key] > 0:
            res[key] = non_uniques[key]
    return res

def change_non_unique_names(dic, non_uniques, new_dic={}):
    for key in dic:
        if key in non_uniques:
                split = key.split('.')
                if len(split) > 1:
                    before_dot = ''.join(split[:len(split)-1])
                    new_key = before_dot + str(non_uniques[key]) + '.' + split[-1]
                else:
                    whole_name = split[0]
                    new_key = whole_name + str(non_uniques[key])
                non_uniques[key] -= 1
        else:
            new_key = key
        
        if type(dic[key]) is dict:
            temp = change_non_unique_names(dic[key], non_uniques, {})
            new_dic[new_key] = temp
        else:
            new_dic[new_key] = dic[key]
    return new_dic  
                        
def create_nodes_and_edges(G, dic, root):
    for key in dic:
        if type(dic[key]) is dict:
            G.add_node(key, label=key)
            G.add_edge(root, key)
            G = create_nodes_and_edges(G, dic[key], key)
        else:
            G.add_node(key, label=dic[key])
            G.add_edge(root, key)
    return G

In [ ]:
chunks_path = '/home/pogorelov/data/'
path_to_save = '/home/pogorelov/gexf_graphs/'


files = []
files_count = 0
portion = 0.1
for filename in os.listdir(chunks_path):
    files.append(filename)
    files_count += 1
shuffle(files)
# num_chunks_to_read = int(float(files_count)*0.1)
# files = files[:num_chunks_to_read]

num_chunks_to_print = int(float(files_count)*portion)
failed_jbls = {}
count_jbl = 0
start = time.time()
for file_ in files:
    try:
        chunk = joblib.load(chunks_path + file_)
        for i, item in enumerate(chunk):
            non_unique_nodes = count_non_unique_names(item[1], {})
            if any(non_unique_nodes):
                graph_dict = change_non_unique_names(item[1], non_unique_nodes, {})
            else:
                graph_dict = item[1]
            G = nx.Graph()
            G.add_node('APK_ROOT', label='APK_ROOT')
            G = create_nodes_and_edges(G, graph_dict, 'APK_ROOT')
            nx.write_gexf(G, path_to_save + item[0] + '.gexf')
        count_jbl += 1
        if count_jbl % num_chunks_to_print == 0:
            print('[%d] chunks are read' % count_jbl)
    except (UnicodeDecodeError, ValueError, EOFError) as e:
        print('%s jbl failed, error - %s' % (file_, e))
        failed_jbls[file_] = e
        print('in total %d failed jbls' % len(failed_jbls))

end = time.time()
    
print('downloading and creating %d gexfs is %0.2f min' % (len(files), (end - start)/60))
print()

_,_,files = os.walk(path_to_save).next()
print('%d files in ' + path_to_save % len(files))

## Labels

In [3]:
path_to_clean = '/Users/grigoriipogorelov/Desktop/KL/v7/clean_g.csv'
path_to_malw = '/Users/grigoriipogorelov/Desktop/KL/v7/malw_g.csv'
path_to_pua = '/Users/grigoriipogorelov/Desktop/KL/v7/pua_g.csv'

clean_g = pd.read_csv(path_to_clean)
malw_g = pd.read_csv(path_to_malw)
pua_g = pd.read_csv(path_to_pua)

clean_g_list = clean_g.iloc[:,-1].tolist()
malw_g_list = malw_g.iloc[:,-1].tolist()
pua_g_list = pua_g.iloc[:,-1].tolist()

len(clean_g_list), len(malw_g_list), len(pua_g_list)

In [ ]:
del clean_g
del malw_g
del pua_g

In [7]:
path_to_gexf = '/Users/grigoriipogorelov/Desktop/KL/gexf_graphs/'
path_to_labels_file = '/Users/grigoriipogorelov/Desktop/KL/labels.Labels'

_,_,files = os.walk(path_to_gexf).next()

with open(path_to_labels_file, 'w') as f:
    for file_ in files:
        clean, malw, pua = False, False, False
        count +=1
        spl = file_.split('.')
        name = spl[0]
        if name in clean_g_list:
            clean = True
        if name in malw_g_list:
            malw = True
        if name in pua_g_list:
            pua = True
            
        if clean and pua:
            f.write(file_+' '+'2'+'\n')
            continue
        if malw and pua:
            f.write(file_+' '+'1'+'\n')
            continue
        if clean:
            f.write(file_+' '+'0'+'\n')
        if malw:
            f.write(file_+' '+'1'+'\n')
        if pua:
            f.write(file_+' '+'2'+'\n')

len(missed), len(files), count

In [17]:
# check=[]
# with open('/Users/grigoriipogorelov/Desktop/KL/labels.Labels') as f:
#     check = f.readlines()
# check = [x.strip() for x in check]
# len(check)

In [3]:
# with open('/Users/grigoriipogorelov/Desktop/KL/clean_g.txt') as f:
#     clean_g_list = f.readlines()
# clean_g_list = [x.strip() for x in clean_g_list]

# with open('/Users/grigoriipogorelov/Desktop/KL/malw_g.txt') as f:
#     malw_g_list = f.readlines()
# malw_g_list = [x.strip() for x in malw_g_list]

# with open('/Users/grigoriipogorelov/Desktop/KL/pua_g.txt') as f:
#     pua_g_list = f.readlines()
# pua_g_list = [x.strip() for x in pua_g_list]